<a href="https://colab.research.google.com/github/tylaar1/PICAR-autopilot/blob/main/Copy_of_BA_cleaned_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# remember to switch to t4 gpu

# Imports

In [ ]:
!ls

In [ ]:
!pip install pandas numpy tensorflow scikit-learn matplotlib


In [ ]:
!pip install pillow  # For image processing


In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# 1) DATA PRE-PROCESSING

a) Load in labels + image file paths

b) combine them into one dataframe

c) EDA - spotted and removed erroneous label (speed = 1.42...)

## `cleaned_df` is the final df with all of this completed

### 1a) load in labels + image file paths

In [ ]:
import os

dir_path = "/Users/jmarsh/Library/Mobile Documents/com~apple~CloudDocs/UoN MCiS/MLiS part 2"

# List all files in the directory
print(os.listdir(dir_path))


In [ ]:
!ls "/Users/jmarsh/Documents/machine-learning-in-science-ii-2025/training_data/training_data"

In [ ]:
labels_file_path = "/Users/jmarsh/Documents/machine-learning-in-science-ii-2025/training_norm.csv"
#labels_file_path = '/content/drive/MyDrive/0. MSc MLiS/google SPRINGrary/Mobile Documents/com~apple~CloudDocs/ SEMESTER/1. PHYS4036 MLiS2/MLiS2 Project/KAGGLEDATAmachine-learning-in-science-ii-2025/training_norm.csv'
labels_df = pd.read_csv(labels_file_path, index_col='image_id')

In [ ]:
#image_folder_path = '/content/drive/MyDrive/0. MSc MLiS/google SPRING SEMESTER/1. PHYS4036 MLiS2/MLiS2 Project/KAGGLEDATAmachine-learning-in-science-ii-2025/training_data/training_data'
image_folder_path = '/Users/jmarsh/Documents/machine-learning-in-science-ii-2025/training_data/training_data'
image_file_paths = [
    os.path.join(image_folder_path, f)
    for f in os.listdir(image_folder_path)
    if f.lower().endswith(('.png', '.jpg', '.jpeg'))
]

image_file_paths.sort(key=lambda x: int(os.path.splitext(os.path.basename(x))[0])) # sorts the files in the right order (1.png, 2.png, 3.png, ...)

imagefilepaths_df = pd.DataFrame(
    image_file_paths,
    columns=['image_file_paths'],
    index=[int(os.path.splitext(os.path.basename(path))[0]) for path in image_file_paths]
)

imagefilepaths_df.index.name = 'image_id'

Checking labels dataframe

In [ ]:
labels_df.head()

Checking image file paths dataframe - as you can see the file paths are ordered correctly (1.png, 2.png, 3.png, ...)

In [ ]:
imagefilepaths_df.head()

### 1b) Combine labels and image file paths into one dataframe

In [ ]:
merged_df = pd.merge(labels_df, imagefilepaths_df, on='image_id', how='inner')
merged_df['speed'] = merged_df['speed'].round(6) # to get rid of floating point errors

In [ ]:
merged_df.head()

In [ ]:
merged_df.loc[3139:3143]

The above cell shows that:

 1) the image files and labels match (see image_id and the number at the end of the file path)

 2) the missing rows in labels_df (image_id: 3141, 3999, 4895, 8285, 10171) have been taken care of

### 1c) EDA

In [ ]:
merged_df.value_counts('speed')

note: imbalance datset

identifying the row with the erroneous speed value

In [ ]:
merged_df[merged_df['speed'] == 1.428571]

we want to remove this row

In [ ]:
cleaned_df = merged_df[merged_df['speed'] != 1.428571]
cleaned_df.loc[3882:3886]

## convert from pandas to tf


In [ ]:
def process_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)  # Use decode_png for PNG images
    image = tf.image.resize(image, (128, 128))  # Resize to uniform shape
    image = image / 255.0  # Normalize pixel values to [0,1]
    return image, label

# Convert DataFrame into a TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((cleaned_df["image_file_paths"], cleaned_df["speed"]))

dataset = dataset.map(process_image, num_parallel_calls=tf.data.AUTOTUNE)

dataset = dataset.cache()
dataset = dataset.shuffle(len(cleaned_df))
dataset = dataset.batch(32)
dataset = dataset.prefetch(tf.data.AUTOTUNE)


for images, labels in dataset.take(1):
    print(images.shape, labels.shape)

In [ ]:
dataset_size = tf.data.experimental.cardinality(dataset).numpy()
train_size = int(0.8 * dataset_size)

# Split into training and test sets
train_dataset = dataset.take(train_size)
validation_dataset = dataset.skip(train_size)

#not using test set as kaggle is our test set

print(f"Train size: {train_size}, Test size: {dataset_size - train_size}")

In [ ]:
for batch in train_dataset.take(1):
    print(batch)

In [ ]:
dropoutrate = 0.2
num_classes = 2
input_shape = (128,128,3)

mbnet =  tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')

model = tf.keras.Sequential([
  mbnet,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dropout(dropoutrate),
  tf.keras.layers.Dense(num_classes, activation='softmax')
])
model.build()

mbnet.trainable = False # freeze the first layers to the imagenet weights

model.summary() # print the model

In [ ]:
with tf.GradientTape() as tape:
  tape.reset() #this resets the gradient tape

In [ ]:
LR = 0.001 #learning rate
optimizer = tf.optimizers.Adam(LR) #adam optimiser

@tf.function
def train_step( model, X , Y):
    with tf.GradientTape() as tape:
        pred = model( X )
        current_loss = tf.reduce_mean(tf.losses.categorical_crossentropy( Y,  pred))
    grads = tape.gradient(current_loss, model.trainable_variables)
    optimizer.apply_gradients( zip( grads , model.trainable_variables) )
    current_accuracy = tf.reduce_mean(tf.metrics.categorical_accuracy(Y, pred))
    return(current_loss, current_accuracy)

In [ ]:
niter = 20

tloss = []
tacc = []
vloss = []
vacc = []

for it in range(niter):
    for image_batch, label_batch in train_dataset:
      #for image, label in zip(image_batch, label_batch):
        #print(image)
        #print(label)
        #print(image.shape, label.shape)

        loss, acc = train_step(model, image_batch, tf.one_hot(tf.cast(label_batch, dtype=tf.int32), depth=2)) #run training


    if it % 10 == 0: #log training metrics
      tf.print('iter: ',it, ', loss: {:.3f}, acc: {:.3f}'.format(loss, acc))
      tloss.append(loss)
      tacc.append(acc)
'we should probably switch to balanced accuracy as eval method due to unbalanced data'
#commented out validation for now as it prints for each batch not each epoch massively slowwing process
'''
    if it % 50 == 0: #log validation metrics
      for val_image, val_label in validation_dataset:
        val_pred = model(val_image)
        val_int=tf.cast(val_label, dtype=tf.int32)
        val_loss = tf.reduce_mean(tf.losses.categorical_crossentropy(tf.one_hot(val_int,depth=2) , val_pred))
        val_acc = tf.reduce_mean(tf.metrics.categorical_accuracy(tf.one_hot(val_int,depth=2) , val_pred))
        tf.print('iter: ',it, ', validation loss: {:.3f}, validation acc: {:.3f}'.format(val_loss, val_acc))
        vloss.append(val_loss)
        vacc.append(val_acc)
'''

In [ ]:
f, axarr = plt.subplots(1,10)

i = 0
for image_batch, label_batch in dataset.take(1):  # Take one batch
    for image in image_batch:  # Iterate through images in the batch
        if i < 10:  # Only display the first 5 images
            print('image shape: ', np.shape(image))
            tf.print('label:', label_batch[i])  # Print label for the corresponding image
            axarr[i].imshow(image)
            axarr[i].axis('off')
            i += 1
        else:
            break  # Stop after displaying 5 images

# add data augmentation steps here

# transfer learning here

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
#installing the extra packages that we need



(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()

train_images, test_images = train_images / 255.0, test_images / 255.0  # Normalize pixel values

# convert labells to the categorical format
num_classes = 10
train_labels = keras.utils.to_categorical(train_labels, num_classes)
test_labels = keras.utils.to_categorical(test_labels, num_classes)

# load pre-trained MobileNetV2 model without top layers
base_model = MobileNetV2(input_shape=(32, 32, 3), include_top=False, weights='imagenet')
base_model.trainable = False

# adding customn layers on top
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# compileing model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# training model
history = model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))

# plot accuracy and loss
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss')
plt.show()

# evaluate on test set
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc:.4f}')


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import cv2
import os

# Paths
image_folder = "/Users/jmarsh/Documents/machine-learning-in-science-ii-2025/training_data/training_data"
label_file = "/Users/jmarsh/Documents/machine-learning-in-science-ii-2025/training_norm.csv"


def load_data(image_folder, label_file):
    images = []
    labels = []

    with open(label_file, 'r') as f:
        # Skips the headder
        next(f)

        for line in f:
            parts = line.strip().split(',')
            image_path = os.path.join(image_folder, parts[0])  # Assuming first column is image filename

            try:
                steering_angle = float(parts[1])
                speed = float(parts[2])  # Assuming third column is speed
            except ValueError:
                print(f"Skipping invalid line: {line.strip()}")  # Skip lines with invalid data
                continue

            # Check if the image exists
            if not os.path.exists(image_path):
                print(f"Image not found: {image_path}")
                continue  # Skip if the image doesn't exist

            img = cv2.imread(image_path)

            # Check if the image was loaded properly
            if img is None:
                print(f"Failed to load image: {image_path}")
                continue  # Skip if the image is invalid

            img = cv2.resize(img, (128, 128))  # are we doing 320 x 320 ?
            img = img / 255.0  # normalize pixel value. also not sure if this is the correct normalisation?

            images.append(img)
            labels.append([steering_angle, speed])

    return np.array(images), np.array(labels)


# Loaddoming images and labels
X, y = load_data(image_folder, label_file)

# Split data into training and validation sets
split = int(0.8 * len(X))
X_train, X_val = X[:split], X[split:]
y_train, y_val = y[:split], y[split:]

# define the CNN model for regression
model = keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(2)  # Output two values: [steering_angle, speed]
])

# Compilee the model for regression
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32
)

# Save the model
model.save("/Users/jmarsh/Documents/machine-learning-in-science-ii-2025/autonomous_driving_model.h5")

print("Model training complete and saved as 'autonomous_driving_model.h5'.")

